In [1]:
from UTILITY_quickstart import *

In [2]:
#Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Base Tao lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


In [3]:
getMatrix(tao, "BEGBC20", "MFFF", print = True);

-1.208438,-4.961819,-0.000000,-0.000000,0.000000,-0.000000
-0.084842,-1.175874,-0.000000,-0.000000,0.000000,-0.000000
-0.000000,-0.000000,-0.638379,6.515289,0.000000,-0.000000
-0.000000,-0.000000,-0.094469,-0.602321,0.000000,-0.000000
0.000000,0.000000,0.000000,0.000000,1.000000,-0.007000
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [4]:
#Load a default configuration
#importedDefaultSettings = loadConfig("setLattice_configs/2024-10-22_oneBunch_baseline.yml")
importedDefaultSettings = loadConfig("setLattice_configs/defaults.yml")

#Initialize tao object
tao = initializeTao(
    #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
    #numMacroParticles = 1e4,
    #csrTF = False
)

#Set lattice to the loaded config
setLattice(tao, **importedDefaultSettings)

getMatrix(tao, "BEGBC20", "MFFF", print = True);

launchTwissCorrection(tao)

baseMatrix = getMatrix(tao, "BEGBC20", "MFFF", print = True);

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
WARNING! No beam file is specified!
Number of macro particles defined by input file


-1.208675,-4.962570,-0.000000,-0.000000,0.000000,0.000012
-0.084965,-1.176201,0.000000,0.000000,0.000000,-0.000001
-0.000000,-0.000000,-0.637675,6.526881,0.000000,-0.000000
-0.000000,-0.000000,-0.094432,-0.601645,0.000000,-0.000000
-0.000003,-0.000021,0.000000,-0.000000,1.000000,-0.007001
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


No evalElement provided. Assuming golden lattice PR10571
Optimization Results:
Optimal Parameters: [0.13755652 0.953558   0.40561699 2.20425982]
Objective Function Value at Optimal Parameters: 3.3948865246987365e-09
Number of Iterations: 235
Converged: True


-1.208675,-4.962570,-0.000000,-0.000000,0.000000,0.000012
-0.084965,-1.176201,0.000000,0.000000,0.000000,-0.000001
-0.000000,-0.000000,-0.637675,6.526881,0.000000,-0.000000
-0.000000,-0.000000,-0.094432,-0.601645,0.000000,-0.000000
-0.000003,-0.000021,0.000000,-0.000000,1.000000,-0.007001
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [5]:
setLattice(tao, **(importedDefaultSettings | {'Q3EkG': 110}))

newMatrix = getMatrix(tao, "BEGBC20", "MFFF", print = True);

-1.310429,-5.082830,-0.000000,-0.000000,0.000000,0.007158
-0.128425,-1.261239,0.000000,0.000000,0.000000,0.000397
-0.000000,-0.000000,-0.692087,5.709292,0.000000,-0.000000
0.000000,-0.000000,-0.095684,-0.655575,0.000000,-0.000000
-0.000399,-0.007012,0.000000,0.000000,1.000000,-0.007487
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [6]:
displayMatrix(  newMatrix - baseMatrix ) 

-0.101754,-0.120260,-0.000000,-0.000000,0.000000,0.007145
-0.043460,-0.085038,-0.000000,-0.000000,0.000000,0.000398
0.000000,0.000000,-0.054411,-0.817590,0.000000,0.000000
0.000000,0.000000,-0.001252,-0.053930,0.000000,0.000000
-0.000397,-0.006991,-0.000000,0.000000,0.000000,-0.000486
0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
def getNewMatrix(params):
    q1, q2, q3, q4, q5, q6 = params
    setLattice(tao, **(importedDefaultSettings | {
        'Q1EkG': q1,
        'Q2EkG': q2,
        'Q3EkG': q3,
        'Q4EkG': q4,
        'Q5EkG': q5,
        'Q6EkG': q6,

    }))

    return( getMatrix(tao, "BEGBC20", "MFFF") )

In [8]:
baselineParams = [ importedDefaultSettings[ele] for ele in ['Q1EkG', 'Q2EkG', 'Q3EkG', 'Q4EkG', 'Q5EkG', 'Q6EkG'] ]
baselineParams

[161.311, -154.229, 110.217, 132.268, -23.373, -142.271]

In [9]:
getNewMatrix( baselineParams )

array([[-1.20867456e+00, -4.96256976e+00, -1.33718758e-17,
        -1.10207470e-16,  0.00000000e+00,  1.23649164e-05],
       [-8.49649366e-02, -1.17620117e+00, -1.90000751e-18,
        -2.33770173e-17,  0.00000000e+00, -1.24861596e-06],
       [-2.38694683e-16, -1.12476605e-15, -6.37675400e-01,
         6.52688129e+00,  0.00000000e+00, -2.74050514e-17],
       [-2.92697655e-18, -1.41951754e-17, -9.44318686e-02,
        -6.01645293e-01,  0.00000000e+00, -8.06887982e-19],
       [-2.55975468e-06, -2.07399729e-05,  1.46759644e-19,
         8.85450066e-18,  1.00000000e+00, -7.00081917e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [16]:
#The objective is to create a transport matrix that is the same as the baseline, except with a larger dispersion term

def objective(params):
    newMatrix = getNewMatrix(params)

    targetMatrix = baseMatrix + [[0,0,0,0,0,0.001],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

    errorMatrix = np.array( newMatrix - targetMatrix )

    error = np.sum(errorMatrix ** 2) 

    #print(error)
    
    return error
    

    


In [17]:
from scipy.optimize import minimize

# Perform optimization using Nelder-Mead
result = minimize(
    objective, 
    baselineParams, #Starting point
    method='Nelder-Mead',
)

                      
print("Optimization Results:")
print(f"Optimal Parameters: {result.x}")
print(f"Objective Function Value at Optimal Parameters: {result.fun}")
print(f"Number of Iterations: {result.nit}")
print(f"Converged: {result.success}")

Optimization Results:
Optimal Parameters: [ 161.41813922 -154.21587792  110.15190023  132.48069395  -23.40370404
 -142.3112049 ]
Objective Function Value at Optimal Parameters: 4.955996671329345e-07
Number of Iterations: 332
Converged: True


In [18]:
proposedParams = result.x

In [19]:
deltas = proposedParams - baselineParams
deltas

array([ 0.10713922,  0.01312208, -0.06509977,  0.21269395, -0.03070404,
       -0.0402049 ])

In [20]:
deltas / max(abs(deltas))

array([ 0.5037248 ,  0.06169467, -0.30607251,  1.        , -0.14435786,
       -0.18902699])

In [21]:
displayMatrix(
getNewMatrix(proposedParams) - baseMatrix
)

-0.000000,0.000000,0.000000,0.000000,0.000000,0.000504
-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000019
-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000
-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000
-0.000019,-0.000497,0.000000,0.000000,0.000000,-0.000047
0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### X Dispersion prime

In [2]:
def objective(params):
    newMatrix = getNewMatrix(params)

    targetMatrix = baseMatrix + [[0,0,0,0,0,0],[0,0,0,0,0,0.01],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

    errorMatrix = np.array( newMatrix - targetMatrix )

    error = np.sum(errorMatrix ** 2) 

    #print(error)
    
    return error

In [3]:
# Perform optimization using Nelder-Mead
result = minimize(
    objective, 
    baselineParams, #Starting point
    method='Nelder-Mead',
)

                      
print("Optimization Results:")
print(f"Optimal Parameters: {result.x}")
print(f"Objective Function Value at Optimal Parameters: {result.fun}")
print(f"Number of Iterations: {result.nit}")
print(f"Converged: {result.success}")

NameError: name 'minimize' is not defined

In [17]:
proposedParams = result.x

In [18]:
deltas = proposedParams - baselineParams
deltas

array([-2.32386764e-04, -4.62280607e-06, -1.13421156e-04,  2.55012031e-04,
       -8.00062266e-05,  4.99558093e-05])

In [19]:
deltas / max(abs(deltas))

array([-0.91127765, -0.0181278 , -0.44476787,  1.        , -0.31373511,
        0.19589589])

In [21]:
displayMatrix(
getNewMatrix(proposedParams) - baseMatrix
)

-0.000021,0.000011,0.000000,0.000000,0.000000,0.000003
-0.000009,-0.000017,-0.000000,-0.000000,0.000000,0.000000
-0.000000,-0.000000,-0.000006,-0.000004,0.000000,0.000000
-0.000000,-0.000000,0.000001,-0.000006,0.000000,0.000000
-0.000000,-0.000003,0.000000,0.000000,0.000000,-0.000000
0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Using bends only instead

In [22]:
#Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Base Tao lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


In [23]:
def getNewMatrix(params):
    b1, b2, b3 = params
    setLattice(tao, **(importedDefaultSettings | {
        'B1EkG': b1,
        'B2EkG': b2,
        'B3EkG': b3,


    }))

    return( getMatrix(tao, "BEGBC20", "MFFF") )

In [24]:
baselineParams = [ importedDefaultSettings[ele] for ele in ['B1EkG', 'B2EkG', 'B3EkG'] ]
baselineParams

[7.533, -10.942, 3.409]

In [25]:
getNewMatrix( baselineParams )

array([[-1.21868291e+00, -4.97352885e+00, -1.48618301e-17,
        -1.22236825e-16,  0.00000000e+00,  6.55003019e-04],
       [-8.91980482e-02, -1.18458137e+00,  9.78815662e-18,
         7.39271849e-17,  0.00000000e+00,  2.46566200e-05],
       [-6.92192800e-17, -4.21118447e-16, -6.18300468e-01,
         6.83251047e+00,  0.00000000e+00, -7.01055271e-18],
       [-3.62284491e-19, -3.65629728e-18, -9.36395716e-02,
        -5.82575406e-01,  0.00000000e+00, -1.03154971e-19],
       [-2.83763896e-05, -6.53273963e-04,  9.79969261e-19,
         1.70298387e-17,  1.00000000e+00, -7.04089420e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [26]:
#The objective is to create a transport matrix that is the same as the baseline, except with a larger dispersion term

def objective(params):
    newMatrix = getNewMatrix(params)

    targetMatrix = baseMatrix + [[0,0,0,0,0,0.001],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

    errorMatrix = np.array( newMatrix - targetMatrix )

    error = np.sum(errorMatrix ** 2) 

    #print(error)
    
    return error
    

In [27]:
from scipy.optimize import minimize

# Perform optimization using Nelder-Mead
result = minimize(
    objective, 
    baselineParams, #Starting point
    method='Nelder-Mead',
)

                      
print("Optimization Results:")
print(f"Optimal Parameters: {result.x}")
print(f"Objective Function Value at Optimal Parameters: {result.fun}")
print(f"Number of Iterations: {result.nit}")
print(f"Converged: {result.success}")

Optimization Results:
Optimal Parameters: [  7.48103038 -10.95999322   3.40039772]
Objective Function Value at Optimal Parameters: 4.935434127213309e-07
Number of Iterations: 60
Converged: True


In [28]:
proposedParams = result.x

In [29]:
deltas = proposedParams - baselineParams
deltas

array([-0.05196962, -0.01799322, -0.00860228])

In [30]:
displayMatrix(
getNewMatrix(proposedParams) - baseMatrix
)

-0.000010,-0.000002,0.000000,0.000000,0.000000,0.000506
-0.000005,-0.000009,-0.000000,-0.000000,0.000000,0.000019
-0.000000,-0.000000,-0.000003,0.000000,0.000000,0.000000
-0.000000,-0.000000,0.000001,-0.000004,0.000000,0.000000
-0.000019,-0.000499,-0.000000,0.000000,0.000000,-0.000000
0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
